In [1]:
import csv, os,sqlite3
import pandas as pd

from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

from dbfuncs import *

In [2]:
working_dir = "/home/srivbane/shared/caringbridge/data/projects/qual-health-journeys/identify_candidate_sites"
valid_classification_sites_filename = os.path.join(working_dir, "valid_classification_sites.txt")
valid_sites_filtered_filename = os.path.join(working_dir, "valid_classification_sites_filtered.txt")

In [3]:
file_name = valid_classification_sites_filename #58000
#file_name = valid_sites_filtered_filename #5000

if os.path.exists(file_name):
    with open(file_name, 'r') as infile:
        valid_sites = [int(line.strip()) for line in infile.readlines() if line.strip() != ""]
else:
    valid_sites = []
    
print(len(valid_sites))

58363


In [4]:
loc = "/panfs/roc/groups/3/srivbane/shared/caringbridge/data/projects/qual-health-journeys/death_analysis/"
df = pd.read_csv(loc+"cleaned_probability.csv")
#df.head()

In [5]:
def author_probability(site_id):
    journal_oids = get_site_journal_oid(site_id)
    
    total = 0
    count = 0
    
    ds = df.loc[df['site_id'] == site_id] 
    
    for x in journal_oids:
        a = ds.loc[ds['journal_oid'] == x] 
        
        if len(a) != 0:
            if a['is_patient'].iloc[0]:
                total += 1
                count += 1
            else:
                count += 1
                
    if len(a) == 0 or a['is_patient'].iloc[0]:
        last_patient = 1
    else:
        last_patient = 0
    
    return [site_id,total,count,last_patient]

In [6]:
results = []

for x in tqdm(range(len(valid_sites))):
    
    results.append(author_probability(valid_sites[x]))

print(len(results))

100%|██████████| 58363/58363 [52:50<00:00, 18.41it/s] 

58363


In [9]:
csvfile = "58363_probability.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(["site_id","patient_authored","total_count","patient_authored_percentage","last_patient_authored"])
    for val in results:
        if len(val) != 5:
            if val[2] == 0:
                percentage = 0
            else:
                percentage = val[1]/val[2]*100
            val.insert(3,percentage)
        writer.writerow(val)

In [11]:
print (results[-10:])

[[1097560, 0, 30, 0.0, 0], [1097940, 8, 8, 100.0, 1], [1097953, 0, 22, 0.0, 0], [1099282, 1, 5, 20.0, 0], [1099418, 0, 15, 0.0, 0], [1099548, 2, 9, 22.22222222222222, 0], [1099652, 3, 15, 20.0, 0], [1099768, 1, 11, 9.090909090909092, 0], [1101066, 2, 14, 14.285714285714285, 0], [1101720, 1, 17, 5.88235294117647, 1]]
